We are trying to predict the formation energies of the molecules using active learning.
We want to use as little data points as possible to train the model but, at the same time, having better accuracy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel
from modAL.models import ActiveLearner
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

Loading our X data: molecular fingerprints(number of counts for each feature) y data: formation energies

In [2]:
df = pd.read_csv('SMILES_energy_patterns.txt', delim_whitespace = True, header = None)

patterns = []

fileinput = open('QM7_300_patterns_v2.txt','r').readlines()
for line in fileinput:
    patterns.append(line.split()[0])

In [3]:
print(patterns)
patterns = np.array(patterns)
print(patterns.shape)

['C', 'N', 'O', 'S', 'c', 'n', 'o', 's', 'C#C', 'C=C', 'CC', 'OC', 'O=C', 'NC', 'N#C', 'N=C', 'ON', 'cc', 'nc', 'sc', 'sn', 'oc', 'on', 'SC', 'S=O', 'COC', 'O=CC', 'OCC', 'OR1CCR1', 'CNC', 'N#CC', 'NCC', 'NR1CCR1', 'C#CC', 'C=CC', 'CCC', 'CR1CCR1', 'NOC', 'ON=C', 'O=CN', 'N=CC', 'OC=C', 'cnc', 'csc', 'ncc', 'scc', 'scn', 'ccc', 'nsc', 'snc', 'NC=C', 'C=NC', 'coc', 'occ', 'ocn', 'noc', 'onc', 'CSC', 'O=SC', 'O=S=O', 'SCC', 'Occ', 'scO', 'Ncc', 'scN', 'SC#C', 'SC=C', 'ncC', 'scC', 'ccC', 'Onc', 'Ocn', 'ocN', 'OCCN', 'C=NOC', 'O=CNC', 'ON=CC', 'OCC#C', 'OCC=C', 'O=CC#C', 'OCCC', 'C=COC', 'COCC', 'O=CCC', 'O=CC=C', 'OR1CCCR1', 'NCC#C', 'NCC=C', 'NCCC', 'CNCC', 'N#CCC', 'N#CC#C', 'N#CC=C', 'NR1CCCR1', 'C#CCC', 'C=CCC', 'CC#CC', 'CC=CC', 'C#CC#C', 'C#CC=C', 'C=CC=C', 'CCCC', 'CR1CCCR1', 'cncc', 'cscc', 'ncsc', 'sccn', 'scnc', 'csnc', 'nccc', 'nscc', 'sccc', 'sncc', 'cccc', 'N=CC#C', 'N=CC=C', 'O=CCN', 'NC=CC', 'NOCC', 'CN=CC', 'O=CN=C', 'N=CCC', 'cocc', 'ncoc', 'occn', 'ocnc', 'conc', 'nocc'

In [4]:
df.iloc[:,2]

0      -417.031
1      -403.695
2      -563.084
3      -711.117
4      -795.364
         ...   
295   -1404.080
296   -1294.540
297   -1556.930
298   -1293.440
299   -1557.390
Name: 2, Length: 300, dtype: float64

In [5]:
y = np.array(df.iloc[:,2])
print(y[np.argmax(y)])
print(y[np.argmin(y)])
print(y.shape)

-403.695
-1599.0
(300,)


In [6]:
#the last pattern is the hydrogen counts
X = np.array(df.iloc[:,3:])

print(X.shape)

(300, 389)


We are using maximum uncertainty method to update our model(query strategy)

In [7]:
#def Epsilon_greedy(regressor,X):
#    epsilon = 0.5
#   n_ep = np.random.random()
#    if n_ep > epsilon:
#        query_idx, X[query_idx] = GP_regression_std(regressor,X)
#       return query_idx, X[query_idx]
#    else:

        
def GP_regression_std(regressor, X):           #Uncertainty Sampling query method
    _, std = regressor.predict(X, return_std=True)
    query_idx = np.argmax(std)                #choosing the instance with max standard deviation
    
    return query_idx, X[query_idx]

Initializing the active learner.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)   #Slitting dataset into training and testing


def Initial_trainingSet(n_initial,X_train,y_train):                          #randomly selecting our initial dataset
    initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
    X_training_initial, y_training_initial = X_train[initial_idx], y_train[initial_idx]
    X_training_initial.reshape(n_initial,-1)
    return X_training_initial, y_training_initial

X_training_initial, y_training_initial = Initial_trainingSet(30,X_train,y_train)







#RBF will serve at fitting the non-linearity between the data and the target.

kernel = RBF(length_scale=1, length_scale_bounds=(1e2, 2e3))

#Gaussian Process Regressor is used for astimator

regressor = ActiveLearner(
    estimator=GaussianProcessRegressor(kernel=kernel),
    query_strategy= GP_regression_std,
    X_training=X_training_initial, y_training=y_training_initial.reshape(-1,1)
)

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [9]:
def Query_teach(n_queries, regressor,X_train,y_train):                          #Query and teach for n times
    for idx in range(n_queries):
        query_idx, query_instance = regressor.query(X_train)
        regressor.teach(X_train[query_idx].reshape(1, -1), y_train[query_idx].reshape(1, -1))

In [10]:
Query_teach(70,regressor,X_train,y_train)                                

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

Validating our model

In [11]:
y_predict_train = regressor.predict(X_train)

Minus_train = y_train-y_predict_train.flatten()
#print(y_train.shape)
RMSE_training = np.sqrt(np.mean(np.square(Minus_train)))
#print(y_train)
#print(y_predict_train.flatten())
print('The RMSE of training set is',RMSE_training)

The RMSE of training set is 3.6273348000433105


In [13]:
y_predict_test = regressor.predict(X_test)

Minus_test = y_test-y_predict_test.flatten()

RMSE_testing = np.sqrt(np.mean(np.square(Minus_test)))
print('The RMSE of testing set is ',RMSE_testing)

The RMSE of testing set is  8.507649813314142


In [14]:
kf = KFold(n_splits=5) # five fold cross validation

test_error = 0

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
 
    X_train_initial, y_train_initial = Initial_trainingSet(50,X_train,y_train)
    
    
    regressor_2 = ActiveLearner(
    estimator=GaussianProcessRegressor(kernel=kernel),
    query_strategy= GP_regression_std,
    X_training=X_train_initial, y_training=y_train_initial.reshape(-1,1)
    )
    
    Query_teach(120,regressor_2,X_train,y_train)
    
    predictY = regressor_2.predict(X_test)
    
    test_error += np.sqrt(np.sum(np.square(y_test-predictY.flatten()))/y_test.size)

test_error = test_error/kf.get_n_splits(X)

print("CV error: ", test_error)

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

...

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni

CV error:  10.903516244029326


C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\wesle\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarni